In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

"""トレーニングデータの読み込み"""
# 長さ128(2.56sec)のトレーニングデータが7521件/21人分
acc_x_train_df=pd.read_csv('./UCI HAR Dataset/train/Inertial Signals/total_acc_x_train.txt', header=None, sep='\s+', na_values='na')
acc_y_train_df=pd.read_csv('./UCI HAR Dataset/train/Inertial Signals/total_acc_y_train.txt', header=None, sep='\s+', na_values='na')
acc_z_train_df=pd.read_csv('./UCI HAR Dataset/train/Inertial Signals/total_acc_z_train.txt', header=None, sep='\s+', na_values='na')

# Dataframeからarrayへ変換
acc_x_train=acc_x_train_df.values
acc_y_train=acc_y_train_df.values
acc_z_train=acc_z_train_df.values

# 3軸を横に並べる
X_train=np.c_[acc_x_train, acc_y_train, acc_z_train]

# 正解ラベル
    # 1 WALKING
    # 2 WALKING_UPSTAIRS
    # 3 WALKING_DOWNSTAIRS
    # 4 SITTING
    # 5 STANDING
    # 6 LAYING
train_label_df=pd.read_csv('./UCI HAR Dataset/train/Inertial Signals/y_train.txt', header=None, sep='\s+', na_values='na')
y_train=train_label_df.values.reshape(-1)
le = LabelEncoder()
y_train = le.fit_transform(y_train)

# 正解ラベルをone hot表現にする
#ohe = OneHotEncoder()
#y_train = ohe.fit_transform(train_label).A 

"""トレーニングデータの読み込み"""
# 長さ128(2.56sec)のトレーニングデータが7521件/21人分
acc_x_test_df=pd.read_csv('./UCI HAR Dataset/test/Inertial Signals/total_acc_x_test.txt', header=None, sep='\s+', na_values='na')
acc_y_test_df=pd.read_csv('./UCI HAR Dataset/test/Inertial Signals/total_acc_y_test.txt', header=None, sep='\s+', na_values='na')
acc_z_test_df=pd.read_csv('./UCI HAR Dataset/test/Inertial Signals/total_acc_z_test.txt', header=None, sep='\s+', na_values='na')

# Dataframeからarrayへ変換
acc_x_test=acc_x_test_df.values
acc_y_test=acc_y_test_df.values
acc_z_test=acc_z_test_df.values

# 3軸を横に並べる
X_test=np.c_[acc_x_test, acc_y_test, acc_z_test]

# 正解ラベル
    # 1 WALKING
    # 2 WALKING_UPSTAIRS
    # 3 WALKING_DOWNSTAIRS
    # 4 SITTING
    # 5 STANDING
    # 6 LAYING
test_label_df=pd.read_csv('./UCI HAR Dataset/test/Inertial Signals/y_test.txt', header=None, sep='\s+', na_values='na')
y_test=test_label_df.values.reshape(-1)
le = LabelEncoder()
y_test = le.fit_transform(y_test)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
 
    def __init__(self, num_classes):
        super(CNN, self).__init__() # nn.Moduleを継承する
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=18, kernel_size=(2,12)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(1,2), stride=(1,2)),
            nn.BatchNorm2d(18)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=18, out_channels=36, kernel_size=(1,12)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(1,2), stride=(1,2)),
            nn.BatchNorm2d(36)
        )
        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels=36, out_channels=24, kernel_size=(1,12)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(1,2), stride=(1,2)),
            nn.BatchNorm2d(24)
        )
        self.full_connection = nn.Sequential(
            nn.Linear(in_features=6*2*24, out_features=1024), # in_featuresは直前の出力ユニット数
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(in_features=1024, out_features=num_classes)
        )
 
 
    # Forward計算の定義
    # 参考：Define by Runの特徴（入力に合わせてForward計算を変更可）
    def forward(self, x):
 
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
 
        # 直前のMaxPoolの出力が2次元（×チャネル数）なので，全結合の入力形式に変換
        # 参考：KerasのFlatten()と同じような処理
        x = x.view(x.size(0), -1)
        y = self.full_connection(x)
        
        return y
 
 
# 学習用関数
def train(loader_train, model_obj, optimizer, loss_fn, device, total_epoch, epoch):
    
    model_obj.train() # モデルを学習モードに変更
 
    # ミニバッチごとに学習
    for data, targets in loader_train:
 
        data = data.to(device) # GPUを使用するため，to()で明示的に指定
        targets = targets.to(device) # 同上
 
        optimizer.zero_grad() # 勾配を初期化
        outputs = model_obj(data) # 順伝播の計算
        loss = loss_fn(outputs, targets) # 誤差を計算
        loss.backward() # 誤差を逆伝播させる
        optimizer.step() # 重みを更新する
    
    print ('Epoch [%d/%d], Loss: %.4f' % (epoch, total_epoch, loss.item()))
 
 
# テスト用関数
def test(loader_test, trained_model, device):
 
    trained_model.eval() # モデルを推論モードに変更
    correct = 0 # 正解率計算用の変数を宣言
 
    # ミニバッチごとに推論
    with torch.no_grad(): # 推論時には勾配は不要
        for data, targets in loader_test:
 
            data = data.to(device) #  GPUを使用するため，to()で明示的に指定
            targets = targets.to(device) # 同上
 
            outputs = trained_model(data) # 順伝播の計算
 
            # 推論結果の取得と正誤判定
            _, predicted = torch.max(outputs.data, 1) # 確率が最大のラベルを取得
            correct += predicted.eq(targets.data.view_as(predicted)).sum() # 正解ならば正解数をカウントアップ
    
    # 正解率を計算
    data_num = len(loader_test.dataset) # テストデータの総数
    print('\nAccuracy: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

In [7]:
# 1. GPUの設定（PyTorchでは明示的に指定する必要がある）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# 2. ハイパーパラメータの設定（最低限の設定）
batch_size = 100
num_classes = 6
epochs = 100

# 5. DataLoaderの作成
from torch.utils.data import TensorDataset, DataLoader

# 5-2. データのフォーマットを変換：PyTorchでの形式 = [画像数，チャネル数，高さ，幅]
X_train = X_train.reshape(-1, 1, 3, 128)
X_test = X_test.reshape(-1, 1, 3 ,128)

# 5-3. PyTorchのテンソルに変換
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)
 
# 5-4. 入力(x)とラベル(y)を組み合わせて最終的なデータを作成
ds_train = TensorDataset(X_train, y_train)
ds_test = TensorDataset(X_test, y_test)
 
# 5-5. DataLoaderを作成
loader_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(ds_test, batch_size=batch_size, shuffle=False)
 
# 6. モデル作成
model = CNN(num_classes=num_classes).to(device)
print(model) # ネットワークの詳細を確認用に表示
 
# 7. 損失関数を定義
loss_fn = nn.CrossEntropyLoss()
 
# 8. 最適化手法を定義（ここでは例としてAdamを選択）
from torch import optim
optimizer = optim.Adam(model.parameters(), lr=0.01)
 
# 9. 学習（エポック終了時点ごとにテスト用データで評価）
print('Begin train')
for epoch in range(1, epochs+1):
    train(loader_train, model, optimizer, loss_fn, device, epochs, epoch)
    test(loader_test, model, device)

cpu
CNN(
  (block1): Sequential(
    (0): Conv2d(1, 18, kernel_size=(2, 12), stride=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(18, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (block2): Sequential(
    (0): Conv2d(18, 36, kernel_size=(1, 12), stride=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (block3): Sequential(
    (0): Conv2d(36, 24, kernel_size=(1, 12), stride=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (full_connection): Sequential(
    (0): Linear(in_features=288, out_features=1024, bias=True)
    (1): ReLU(

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\shuns\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-f10dda05a113>", line 45, in <module>
    train(loader_train, model, optimizer, loss_fn, device, epochs, epoch)
  File "<ipython-input-2-b14b96700c7d>", line 65, in train
    loss.backward() # 誤差を逆伝播させる
  File "C:\Users\shuns\Anaconda3\lib\site-packages\torch\tensor.py", line 107, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "C:\Users\shuns\Anaconda3\lib\site-packages\torch\autograd\__init__.py", line 93, in backward
    allow_unreachable=True)  # allow_unreachable flag
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\shuns\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2018, in showtraceback
    stb = value._ren

KeyboardInterrupt: 